In [1]:
import pandas as pd
from pathlib import Path

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import joblib
from sklearn.model_selection import TimeSeriesSplit

# all the functions from helpers.py
from helpers import *

In [2]:
subject = 1
video = 1
scenario = 'scenario_1'
annotation_file = f'../data/raw/{scenario}/train/annotations/sub_{subject}_vid_{video}.csv'
physiology_file = f'../data/raw/{scenario}/train/physiology/sub_{subject}_vid_{video}.csv'
df_annotations = pd.read_csv(annotation_file)
df_physiology = pd.read_csv(physiology_file)
X, y,numeric_column_indices, categorical_column_indices = preprocess(df_physiology, df_annotations, window_size=100, step=5, resample_factor=10)
model = SVR
hyperparameters = {'kernel': 'linear', 'C': 1, 'epsilon': 0.1}

rmse_values, average_rmse = time_series_cross_validation_with_hyperparameters(X, y, model, hyperparameters, n_splits=5, n_jobs=-1,  numeric_column_indices= numeric_column_indices, categorical_column_indices= categorical_column_indices)
print("Average Root Mean Squared Error:", average_rmse)

Error in sys.excepthook:
Traceback (most recent call last):
  File "/Users/nicobruno/opt/anaconda3/envs/mne/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 1934, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'ValueError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/nicobruno/opt/anaconda3/envs/mne/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 1936, in showtraceback
    stb = self.InteractiveTB.structured_traceback(etype,
  File "/Users/nicobruno/opt/anaconda3/envs/mne/lib/python3.9/site-packages/IPython/core/ultratb.py", line 1105, in structured_traceback
    return FormattedTB.structured_traceback(
  File "/Users/nicobruno/opt/anaconda3/envs/mne/lib/python3.9/site-packages/IPython/core/ultratb.py", line 999, in structured_traceback
    return VerboseTB.structured_traceback(
  File "/Users/nicobruno/opt

Run all subjects

In [18]:
annotations_folder = '../data/raw/scenario_1/train/annotations/'
physiology_folder = '../data/raw/scenario_1/train/physiology/'
zipped_files = zip_csv_files(annotations_folder, physiology_folder)
results = {}

for kernel, C, epsilon in hyperparameters:
    print(f"Testing hyperparameters: kernel={kernel}, C={C}, epsilon={epsilon}")
    
    rmse_values = []
    for annotation_file, physiology_file in zipped_files:
        df_annotations = pd.read_csv(annotation_file)
        df_physiology = pd.read_csv(physiology_file)
        X, y = preprocess(df_physiology, df_annotations)
        
        average_rmse = time_series_cross_validation_with_hyperparameters(X, y, kernel, C, epsilon)
        rmse_values.append(average_rmse)
    
    overall_average_rmse = np.mean(rmse_values)
    results[(kernel, C, epsilon)] = overall_average_rmse
    print(f"Overall average root mean squared error for hyperparameters: {overall_average_rmse}")


Testing hyperparameters: kernel=linear, C=0.1, epsilon=0.1
Average Root Mean Squared Error: 0.2702716384258762
Average Root Mean Squared Error: 0.5052056106584261
Average Root Mean Squared Error: 0.990096087127586
Average Root Mean Squared Error: 0.903830011491374
Average Root Mean Squared Error: 0.30243318615073544
Average Root Mean Squared Error: 0.3044765302807789
Average Root Mean Squared Error: 0.5494290520773577
Average Root Mean Squared Error: 0.08521210521222458
Average Root Mean Squared Error: 0.3142041068364454
Average Root Mean Squared Error: 0.8565143279291009
Average Root Mean Squared Error: 0.2396771837061164
Average Root Mean Squared Error: 0.3630030092708271
Average Root Mean Squared Error: 0.09050540193196717
Average Root Mean Squared Error: 0.4642052999936322
Average Root Mean Squared Error: 0.07068191087837024
Average Root Mean Squared Error: 0.1426442724621911
Average Root Mean Squared Error: 0.44348677950131216
Average Root Mean Squared Error: 0.26918368969388873
A